TRAINED IN KAGGLE NOTEBOOK

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0067.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0153.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0105.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0018.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0050.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0029.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0195.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0051.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0074.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0142.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0149.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0121.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0194.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0003.png
/kaggl

Imports

In [2]:
import cv2
import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

Load and Preprocess

In [3]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        # Check for image files
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (128, 128))  # Resize to (128, 128)
                img = img / 255.0  # Normalize pixel values
                images.append(img)  
                labels.append(label)
            else:
                print(f"Could not read image: {filename}")
    return images, labels


X = []
y = []

# Define the path to your dataset
dataset_path = '/kaggle/input/leapgestrecog/leapGestRecog'

# Define gesture labels (update as needed)
gesture_labels = {
    '01_palm': 0,
    '02_l': 1,
    '03_fist': 2,
    '04_fist_moved': 3,
    '05_thumb': 4,
    '06_index': 5,
    '07_ok': 6,
    '08_palm_moved': 7,
    '09_c': 8,
    '10_down': 9
}

# Iterate through subject folders (00, 01, etc.)
for subject in os.listdir(dataset_path):
    subject_path = os.path.join(dataset_path, subject)
    if os.path.isdir(subject_path):
        print(f"Processing subject: {subject}")
        # Iterate through gesture folders (01_palm, 02_l, etc.)
        for gesture_folder_name in os.listdir(subject_path):
            gesture_label = gesture_labels.get(gesture_folder_name, None)
            if gesture_label is not None:
                gesture_folder = os.path.join(subject_path, gesture_folder_name)
                print(f"Loading images from: {gesture_folder}")
                images, labels = load_images_from_folder(gesture_folder, gesture_label)
                X.extend(images)
                y.extend(labels)

X = np.array(X)
y = np.array(y)

print(f"Loaded {len(X)} images.")


Processing subject: 07
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/07/02_l
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/07/04_fist_moved
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/07/09_c
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/07/10_down
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/07/06_index
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/07/08_palm_moved
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/07/07_ok
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/07/05_thumb
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/07/01_palm
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/07/03_fist
Processing subject: 05
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/05/02_l
Loading images from: /kaggle/input/leapgestrecog/leapGestRecog/05/04_fist_moved
Loading images from: /kaggle/input/leapgestrecog/leapGest

Prepare Pytorch data set

In [4]:
class HandGestureDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = torch.tensor(self.images[idx], dtype=torch.float32) 
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label


# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = HandGestureDataset(X_train, y_train)
val_dataset = HandGestureDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


CNN model

In [5]:
class GestureRecognitionModel(nn.Module):
    def __init__(self):
        super(GestureRecognitionModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, len(gesture_labels))  
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)  
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = GestureRecognitionModel()
print(model)


GestureRecognitionModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=65536, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


Train

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
# Assuming you have your DataLoader set up
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Ensure the images have the correct shape for the model
        images = images.unsqueeze(1).to(device)  
        labels = labels.to(device)  

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 0.0086
Epoch [2/10], Loss: 0.0025
Epoch [3/10], Loss: 0.0023
Epoch [4/10], Loss: 0.0009
Epoch [5/10], Loss: 0.0000
Epoch [6/10], Loss: 0.0000
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000


Evaluate

In [7]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images = images.unsqueeze(1).to(device)  
        labels = labels.to(device)

        outputs = model(images)  
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')


Validation Accuracy: 99.97%


In [8]:
# Save the model
model_save_path = 'gesture_recognition_model.pth'  
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to gesture_recognition_model.pth


In [9]:
torch.save(model.state_dict(), 'gesture_recognition_model2.pth')  # Save only the state dict


Predict Test

In [10]:
# Create an inverse mapping for gesture labels
inverse_gesture_labels = {v: k for k, v in gesture_labels.items()}

def predict_gesture(image_path):
    # Check if the image file exists
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise FileNotFoundError(f"Image not found at {image_path}")
    except Exception as e:
        print(e)
        return None

    image = cv2.resize(image, (128, 128))
    image = image / 255.0
    image = torch.tensor(image, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  
    image = image.to(device)

    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)

    predicted_index = predicted.item()
    print(f'Predicted Index: {predicted_index}')

    # Check if the predicted index is valid
    if predicted_index not in inverse_gesture_labels:
        print(f'Invalid predicted index: {predicted_index}. It should be between 0 and {len(gesture_labels)-1}.')
        return None

    gesture = inverse_gesture_labels[predicted_index]
    return gesture

# Test prediction
test_image = '/kaggle/input/leapgestrecog/leapGestRecog/09/10_down/frame_09_10_0002.png'
predicted_gesture = predict_gesture(test_image)
if predicted_gesture is not None:
    print(f'Predicted Gesture: {predicted_gesture}')


Predicted Index: 9
Predicted Gesture: 10_down
